<a href="https://colab.research.google.com/github/NedKost/MS-AAI-501-Team-Project/blob/main/Random_ForestFlight_Failed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

file_path = '/Users/anovayoungers/Downloads/flight_data.csv'
df = pd.read_csv(file_path)


In [ ]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17412876 entries, 0 to 17412875
Data columns (total 37 columns):
 #   Column                           Dtype  
---  ------                           -----  
 0   Year                             int64  
 1   Quarter                          int64  
 2   Month                            int64  
 3   DayofMonth                       int64  
 4   DayOfWeek                        int64  
 5   FlightDate                       object 
 6   Reporting_Airline                object 
 7   Tail_Number                      object 
 8   Flight_Number_Reporting_Airline  int64  
 9   Origin                           object 
 10  Dest                             object 
 11  DepTime                          float64
 12  DepDelay                         float64
 13  TaxiOut                          float64
 14  WheelsOff                        float64
 15  WheelsOn                         float64
 16  TaxiIn                           float64
 17  CRSArr

In [ ]:

df_cleaned = df.dropna()

print("Shape of the cleaned dataset:", df_cleaned.shape)


Shape of the cleaned dataset: (2813734, 37)


In [ ]:
# List of selected features
selected_features = [
    'Reporting_Airline', 'Origin', 'Dest', 'DepTime', 'DepDelay', 'TaxiOut',
    'WheelsOff', 'WheelsOn', 'TaxiIn', 'AirTime', 'Distance', 'CarrierDelay',
    'SecurityDelay', 'CRSArrTime', 'Full-time', 'Part-time',
]

# Select the features and target variable
df_selected = df[selected_features + ['ArrDel15']]


In [ ]:
import pandas as pd

# Define the number of top categories to keep
top_n_reporting_airlines = 10

# Calculate the frequency of each 'Reporting_Airline'
airline_counts = df['Reporting_Airline'].value_counts()

# Get the top N categories by frequency
top_reporting_airlines = airline_counts.head(top_n_reporting_airlines).index

# Replace all other categories with 'Other'
df.loc[~df['Reporting_Airline'].isin(top_reporting_airlines), 'Reporting_Airline'] = 'Other'


In [ ]:
# Define the number of top categories to keep
top_n_origin_dest = 15

# Calculate the frequency of each 'Origin' and 'Dest'
origin_counts = df['Origin'].value_counts()
dest_counts = df['Dest'].value_counts()

# Get the top N categories by frequency for 'Origin'
top_origin = origin_counts.head(top_n_origin_dest).index

# Get the top N categories by frequency for 'Dest'
top_dest = dest_counts.head(top_n_origin_dest).index

# Replace all other categories with 'Other' for 'Origin'
df.loc[~df['Origin'].isin(top_origin), 'Origin'] = 'Other'

# Replace all other categories with 'Other' for 'Dest'
df.loc[~df['Dest'].isin(top_dest), 'Dest'] = 'Other'


In [ ]:
# Remove "Other" category from Reporting_Airline
top_reporting_airlines = df['Reporting_Airline'].value_counts().nlargest(10).index
df = df[df['Reporting_Airline'].isin(top_reporting_airlines)]

# Remove "Other" category from Origin
top_origin_airports = df['Origin'].value_counts().nlargest(15).index
df = df[df['Origin'].isin(top_origin_airports)]

# Remove "Other" category from Dest
top_dest_airports = df['Dest'].value_counts().nlargest(15).index
df = df[df['Dest'].isin(top_dest_airports)]


In [ ]:
from sklearn.model_selection import train_test_split

# Define the features (X) and target variable (y)
X = df[selected_features]
y = df['ArrDel15']

# Split the data into training and testing sets (70% training, 30% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Check the dimensions of the training and testing sets
print("Training set shape (X, y):", X_train.shape, y_train.shape)
print("Testing set shape (X, y):", X_test.shape, y_test.shape)



Training set shape (X, y): (11327760, 16) (11327760,)
Testing set shape (X, y): (4854755, 16) (4854755,)


In [ ]:
# Import the necessary library for one-hot encoding
from sklearn.preprocessing import OneHotEncoder

# Create an instance of the OneHotEncoder
encoder = OneHotEncoder(sparse=False, drop='first')

# Fit and transform the encoder on the training data for 'Reporting_Airline', 'Origin', and 'Dest'
X_train_encoded = encoder.fit_transform(X_train[['Reporting_Airline', 'Origin', 'Dest']])
X_test_encoded = encoder.transform(X_test[['Reporting_Airline', 'Origin', 'Dest']])

# Get the feature names after one-hot encoding
feature_names = encoder.get_feature_names_out(input_features=['Reporting_Airline', 'Origin', 'Dest'])

# Create DataFrames with the encoded data and feature names
X_train_encoded_df = pd.DataFrame(X_train_encoded, columns=feature_names)
X_test_encoded_df = pd.DataFrame(X_test_encoded, columns=feature_names)

# Drop the original categorical columns from the original DataFrames
X_train.drop(columns=['Reporting_Airline', 'Origin', 'Dest'], inplace=True)
X_test.drop (columns=['Reporting_Airline', 'Origin', 'Dest'], inplace=True)

# Concatenate the encoded DataFrames with the original DataFrames
X_train = pd.concat([X_train, X_train_encoded_df], axis=1)
X_test = pd.concat([X_test, X_test_encoded_df], axis=1)


/Users/anovayoungers/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [ ]:
# Drop NaN values from the training set
X_train.dropna(inplace=True)
y_train = y_train[X_train.index]

# Drop NaN values from the testing set
X_test.dropna(inplace=True)
y_test = y_test[X_test.index]


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
# Create the Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Fit the model to the training data
rf_classifier.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = rf_classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

Test Accuracy: 100.00%


This is some series overfitting. To address this let's start by defining the hyperparameter grid and performing grid search with cross-validation. We'll also consider reducing the number of trees if necessary.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

# Define the hyperparameter grid to search
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

rf_classifier = RandomForestClassifier(random_state=42)

# Grid Search with cross-validation
grid_search = GridSearchCV(estimator=rf_classifier, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)

# Use the best hyperparameters to train the final model
best_rf_classifier = RandomForestClassifier(random_state=42, **best_params)
best_rf_classifier.fit(X_train, y_train)

# Make predictions on test data
y_pred = best_rf_classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy:", accuracy)



Fitting 3 folds for each of 216 candidates, totalling 648 fits
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=50; total time=   6.7s
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=50; total time=   6.7s
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   6.8s
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   6.9s
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=50; total time=   6.9s
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   7.2s
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=  11.0s
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time

In [ ]:
# Drop NaN values from the training set
X_train.dropna(inplace=True)
y_train = y_train[X_train.index]

# Drop NaN values from the testing set
X_test.dropna(inplace=True)
y_test = y_test[X_test.index]

In [ ]:
from sklearn.model_selection import train_test_split

# Shuffle the dataset randomly
df_shuffled = df.sample(frac=1, random_state=42)

# Split the data into features (X) and the target variable (y)
X = df_shuffled[selected_features]
y = df_shuffled['ArrDel15']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

(X_train.shape, y_train.shape), (X_test.shape, y_test.shape)


(((11327760, 16), (11327760,)), ((4854755, 16), (4854755,)))

Lets redo the model, creating an entirely new Random Forest Model with new hyperparameters.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Categorical columns to be (re)one-hot encoded
categorical_columns = ['Reporting_Airline', 'Origin', 'Dest']

# Create a ColumnTransformer to apply one-hot encoding to categorical columns
preprocessor = ColumnTransformer(
    transformers=[('cat', OneHotEncoder(), categorical_columns)],
    remainder='passthrough'  # Keep other columns as-is
)

rf_classifier_new = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(
        n_estimators=100,
        criterion='gini',  # Split criterion (Gini or entropy, I'll go with Gini for now)
        max_depth=None,  # Maximum depth of the tree (no limit)
        min_samples_split=2,
        min_samples_leaf=1,
        max_features='sqrt',
        bootstrap=True,
        random_state=42,
        n_jobs=-1  # Number of CPU cores to use for parallelization (-1 = using all available)
    ))
])

rf_classifier_new.fit(X_train, y_train)

y_pred_new = rf_classifier_new.predict(X_test)



In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

accuracy = accuracy_score(y_test, y_pred_new)
print(f'Test Accuracy: {accuracy * 100:.2f}%')

class_report = classification_report(y_test, y_pred_new)
print('Classification Report:\n', class_report)

conf_matrix = confusion_matrix(y_test, y_pred_new)
print('Confusion Matrix:\n', conf_matrix)


Test Accuracy: 100.00%
Classification Report:
               precision    recall  f1-score   support

         1.0       1.00      1.00      1.00    767082

    accuracy                           1.00    767082
   macro avg       1.00      1.00      1.00    767082
weighted avg       1.00      1.00      1.00    767082

Confusion Matrix:
 [[767082]]


Checking for Data Leakage

In [ ]:
# Comparing unique identifiers in X_train and X_test
train_unique_identifiers = set(X_train['DepDelay'])
test_unique_identifiers = set(X_test['DepDelay'])

# Check for common elements between the sets
common_identifiers = train_unique_identifiers.intersection(test_unique_identifiers)

if len(common_identifiers) == 0:
    print("No data leakage detected. There are no common identifiers between training and testing sets.")
else:
    print("Data leakage detected. There are common identifiers between training and testing sets.")


Data leakage detected. There are common identifiers between training and testing sets.


I'm not entirely sure why I tested this, it is pretty clear there would be shared values between the sets. I'll leave this code in for education purposes.

Let's redo the model, now using SMOTE to over-sample the minority (1) in our target variable.

In [ ]:
import pandas as pd

file_path = '/Users/anovayoungers/Downloads/flight_data.csv'
df = pd.read_csv(file_path)

In [ ]:
selected_features = ['Reporting_Airline', 'Origin', 'Dest', 'DepTime', 'AirTime', 'Distance', 'CarrierDelay', 'SecurityDelay', 'CRSElapsedTime', 'Full-time', 'Part-time']

df_selected = df[selected_features + ['ArrDel15']]

df_cleaned = df_selected.dropna(subset=selected_features)

df_cleaned.shape


(2813734, 12)

In [ ]:
import pandas as pd

# Define the number of top categories to keep
top_n_reporting_airlines = 10

# Calculate the frequency of each 'Reporting_Airline'
airline_counts = df_cleaned['Reporting_Airline'].value_counts()

# Get the top N categories by frequency
top_reporting_airlines = airline_counts.head(top_n_reporting_airlines).index

# Replace all other categories with 'Other'
df_cleaned.loc[~df_cleaned['Reporting_Airline'].isin(top_reporting_airlines), 'Reporting_Airline'] = 'Other'

# Define the number of top categories to keep
top_n_origin_dest = 15

# Calculate the frequency of each 'Origin' and 'Dest'
origin_counts = df_cleaned['Origin'].value_counts()
dest_counts = df_cleaned['Dest'].value_counts()

# Get the top N categories by frequency for 'Origin'
top_origin = origin_counts.head(top_n_origin_dest).index

# Get the top N categories by frequency for 'Dest'
top_dest = dest_counts.head(top_n_origin_dest).index

# Replace all other categories with 'Other' for 'Origin'
df_cleaned.loc[~df_cleaned['Origin'].isin(top_origin), 'Origin'] = 'Other'

# Replace all other categories with 'Other' for 'Dest'
df_cleaned.loc[~df_cleaned['Dest'].isin(top_dest), 'Dest'] = 'Other'

# Remove "Other" category from Reporting_Airline
top_reporting_airlines = df_cleaned['Reporting_Airline'].value_counts().nlargest(10).index
df_cleaned = df_cleaned[df_cleaned['Reporting_Airline'].isin(top_reporting_airlines)]

# Remove "Other" category from Origin
top_origin_airports = df_cleaned['Origin'].value_counts().nlargest(15).index
df_cleaned = df_cleaned[df_cleaned['Origin'].isin(top_origin_airports)]

# Remove "Other" category from Dest
top_dest_airports = df_cleaned['Dest'].value_counts().nlargest(15).index
df_cleaned = df_cleaned[df_cleaned['Dest'].isin(top_dest_airports)]


In [ ]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

categorical_features = ['Reporting_Airline', 'Origin', 'Dest']

encoder = OneHotEncoder(drop='first', sparse=False)

encoded_features = encoder.fit_transform(df_cleaned[categorical_features])

encoded_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(categorical_features))

df_encoded = pd.concat([encoded_df, df_cleaned.drop(columns=categorical_features)], axis=1)

df_encoded.shape


/Users/anovayoungers/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


(5005620, 46)

In [ ]:
unique_values = df_encoded['ArrDel15'].unique()
print(unique_values)


[nan  1.]


In [ ]:
print(df_encoded['ArrDel15'].head(16))


0     NaN
1     NaN
2     NaN
3     NaN
4     NaN
5     NaN
6     NaN
7     NaN
8     NaN
9     NaN
10    NaN
11    NaN
12    NaN
13    NaN
14    1.0
15    NaN
Name: ArrDel15, dtype: float64


In [ ]:
df_encoded['ArrDel15'].fillna(0, inplace=True)


In [ ]:
print(df_encoded['ArrDel15'].head(16))

0     0.0
1     0.0
2     0.0
3     0.0
4     0.0
5     0.0
6     0.0
7     0.0
8     0.0
9     0.0
10    0.0
11    0.0
12    0.0
13    0.0
14    1.0
15    0.0
Name: ArrDel15, dtype: float64


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Separate features and target variable
X = df_encoded.drop(columns=['ArrDel15'])
y = df_encoded['ArrDel15']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)



In [ ]:
unique_values = df_encoded['ArrDel15'].unique()
print(unique_values)

[0. 1.]


In [ ]:
from imblearn.over_sampling import ADASYN

X_train.dropna(inplace=True)
y_train = y_train[X_train.index]

X_test.dropna(inplace=True)
y_test = y_test[X_test.index]



In [ ]:
# Initialize the ADASYN object
adasyn = ADASYN(random_state=42)

# Resample the training data
X_resampled, y_resampled = adasyn.fit_resample(X_train, y_train)

ValueError: The target 'y' needs to have more than 1 class. Got 1 class instead

In [ ]:
# Initialize the SMOTE object
smote = SMOTE(random_state=42)

# Resample the training data
X_resampled, y_resampled = smote.fit_resample(X, y)

# Initialize the Random Forest Classifier
rf_classifier = RandomForestClassifier(
    n_estimators=100,  # Number of trees in the forest
    criterion='gini',  # Split criterion (Gini or entropy)
    max_depth=None,  # Maximum depth of the tree (no limit)
    min_samples_split=2,  # Minimum number of samples required to split an internal node
    min_samples_leaf=1,  # Minimum number of samples required to be at a leaf node
    max_features='sqrt',  # Number of features to consider when looking for the best split
    bootstrap=True,  # Bootstrap samples when building trees
    random_state=42,
    n_jobs=-1  # Number of CPU cores to use for parallelization (-1 = using all available)
)

# Define cross-validation strategy (10-fold cross-validation)
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Perform cross-validation and calculate mean accuracy and standard deviation
cross_val_scores = cross_val_score(
    rf_classifier,
    X_resampled,
    y_resampled,
    cv=cv,
    scoring=make_scorer(accuracy_score)
)

mean_accuracy = cross_val_scores.mean()
std_accuracy = cross_val_scores.std()

mean_accuracy, std_accuracy

ValueError: Input X contains NaN.
SMOTE does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
unique_values = df_encoded['ArrDel15'].unique()
print(unique_values)

[0. 1.]


In [ ]:
import numpy as np

# Add a small amount of noise to the 0 values in y
noise = np.random.normal(0, 0.01, len(y))
y = y + noise


In [ ]:
from imblearn.over_sampling import ADASYN


# Separate features and target variable
X = df_encoded.drop(columns=['ArrDel15'])
y = df_encoded['ArrDel15']

adasyn = ADASYN(random_state=42)
X_resampled, y_resampled = adasyn.fit_resample(X, y)



ValueError: The target 'y' needs to have more than 1 class. Got 1 class instead

In [ ]:
class_distribution = y.value_counts()
print(class_distribution)


1.0    228784
Name: ArrDel15, dtype: int64


In [ ]:
# Initialize the Random Forest Classifier
rf_classifier = RandomForestClassifier(
    n_estimators=100,  # Number of trees in the forest
    criterion='gini',  # Split criterion (Gini or entropy)
    max_depth=None,  # Maximum depth of the tree (no limit)
    min_samples_split=2,  # Minimum number of samples required to split an internal node
    min_samples_leaf=1,  # Minimum number of samples required to be at a leaf node
    max_features='sqrt',  # Number of features to consider when looking for the best split
    bootstrap=True,  # Bootstrap samples when building trees
    random_state=42,
    n_jobs=-1  # Number of CPU cores to use for parallelization (-1 = using all available)
)

# Define cross-validation strategy (10-fold cross-validation)
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Perform cross-validation and calculate mean accuracy and standard deviation
cross_val_scores = cross_val_score(
    rf_classifier,
    X_resampled,
    y_resampled,
    cv=cv,
    scoring=make_scorer(accuracy_score)
)

mean_accuracy = cross_val_scores.mean()
std_accuracy = cross_val_scores.std()

mean_accuracy, std_accuracy